In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

TypeError: 'int' object is not iterable

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
import tensorflow as tf

# Load the dataset
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())

with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Function to label the sentiment
def label_sentiment(text):
    words = set(text.lower().split())
    positive_count = len(words.intersection(positive_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > positive_count:
        return 2  # Depressed
    elif positive_count > 0:
        return 1  # Positive
    else:
        return 0  # Neutral/Negative

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Tokenize the text
max_words = 670000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Convert to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train_encoded, num_classes=3)
y_test_cat = tf.keras.utils.to_categorical(y_test_encoded, num_classes=3)

# Define the model
embedding_dim = 200

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
print(model.summary())

# Train the model
model.fit(X_train_pad, y_train_cat,
                    epochs=4,
                    batch_size=32,
                    validation_split=0.1,
                    verbose=1)
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test_cat)
print(f'Test Accuracy: {accuracy:.4f}')

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_12 (Conv1D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_10         │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/4


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 134000000 elements. This may consume a large amount of memory.
  warnings.warn(


385/385 ━━━━━━━━━━━━━━━━━━━━ 98s 249ms/step - accuracy: 0.7143 - loss: 0.6855 - val_accuracy: 0.9306 - val_loss: 0.2184
Epoch 2/4
385/385 ━━━━━━━━━━━━━━━━━━━━ 105s 273ms/step - accuracy: 0.9485 - loss: 0.1629 - val_accuracy: 0.9518 - val_loss: 0.1667
Epoch 3/4
385/385 ━━━━━━━━━━━━━━━━━━━━ 106s 275ms/step - accuracy: 0.9840 - loss: 0.0593 - val_accuracy: 0.9466 - val_loss: 0.2080
Epoch 4/4
385/385 ━━━━━━━━━━━━━━━━━━━━ 107s 279ms/step - accuracy: 0.9937 - loss: 0.0219 - val_accuracy: 0.9598 - val_loss: 0.2238
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9546 - loss: 0.2869
Test Accuracy: 0.9523


In [57]:
loss, accuracy = model.evaluate(X_test_pad, y_test_cat, verbose=0)
print(f"Test accuracy: {accuracy:.2f}")

Test accuracy: 0.95


In [48]:
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a neutral/negative mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
    "I feel so hopeless and sad all the time.",
    "I'm excited about my new job and looking forward to the future.",
    "I don't know if life is worth living anymore.",
    "I had a great day with my friends and family.",
    "This weather is terrible, and I'm having a bad day.",
    "I want to kill myself",
    "I'm feeling a bit down today, but I'm sure things will get better.",
     "I feel like nothing is going right for me today.",
"I just can't seem to focus on anything right now.",
"I'm really struggling to keep up, and it's frustrating.",
"Today feels like a complete waste; I can’t get anything done.",
"I'm overwhelmed, and it's dragging my mood down.",
"I feel stuck in a rut, and it’s wearing me out.",
"Everything I touch seems to go wrong today.",
"I'm feeling really down and unmotivated.",
"I just don't have the energy for anything today.",
"It's one of those days where nothing feels worth the effort."
]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
model.save('sentiment_cnn_model.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Model and tokenizer saved successfully.")

# To load the model and tokenizer later, you can use:
# loaded_model = tf.keras.models.load_model('sentiment_cnn_model.h5')
# with open('tokenizer.pickle', 'rb') as handle:
#     loaded_tokenizer = pickle.load(handle)

Text: I feel so hopeless and sad all the time.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Prediction: The user might be depressed. Confidence: 1.00

Text: I'm excited about my new job and looking forward to the future.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I don't know if life is worth living anymore.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I had a great day with my friends and family.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: This weather is terrible, and I'm having a bad day.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Prediction: The user might be depressed. Confidence: 0.99

Text: I want to kill myself
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user might be depressed. Confidence: 1.00

Text: I'm feeling a bit down today, but I'm sure things will get better.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/st

Prediction: The user is in a positive mood. Confidence: 1.00

Model and tokenizer saved successfully.


In [59]:
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a neutral/negative mood. Confidence: {confidence:.2f}"
examples = [
        "I feel so hopeless and sad all the time.",
        "I'm excited about my new job and looking forward to the future.",
        "I don't know if life is worth living anymore.",
        "I had a great day with my friends and family.",
        "This weather is terrible, and I'm having a bad day.",
        "I'm feeling a bit down today, but I'm sure things will get better.",
        "I'm really proud of what I accomplished today.",
        "Sometimes I wonder if I'll ever find my way.",
        "I had a productive meeting, but I still have a lot on my plate.",
        "Today was just another ordinary day.",
        "I'm thrilled about my upcoming vacation!",
        "I'm feeling overwhelmed by everything that’s happening.",
        "I'm content with where I am in life right now.",
        "I can't shake off this feeling of dread.",
        "I had a fun time at the park with my friends last weekend."
    ]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
model.save('sentiment_cnn_model.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Model and tokenizer saved successfully.")

Text: I feel so hopeless and sad all the time.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction: The user might be depressed. Confidence: 1.00

Text: I'm excited about my new job and looking forward to the future.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I don't know if life is worth living anymore.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I had a great day with my friends and family.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: This weather is terrible, and I'm having a bad day.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Prediction: The user might be depressed. Confidence: 0.99

Text: I'm feeling a bit down today, but I'm sure things will get better.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I'm really proud of what I accomplished today.
1

Prediction: The user is in a positive mood. Confidence: 0.99

Model and tokenizer saved successfully.


In [ ]:
Below this it is only expertiment tunning the model,, 

In [ ]:
testing and tunning only

In [17]:
def predict_sentiment(text):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    predicted_class = np.argmax(prediction)
    confidence = prediction[0][predicted_class]
    
    if predicted_class == 1:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 2:
        return f"The user is in a negative mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
   "I feel like nothing is going right for me today.",
"I just can't seem to focus on anything right now.",
"I'm really struggling to keep up, and it's frustrating.",
"Today feels like a complete waste; I can’t get anything done.",
"I'm overwhelmed, and it's dragging my mood down.",
"I feel stuck in a rut, and it’s wearing me out.",
"Everything I touch seems to go wrong today.",
"I'm feeling really down and unmotivated.",
"I just don't have the energy for anything today.",
"It's one of those days where nothing feels worth the effort."
]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example)}\n")

# Save the model
model.save('sentiment_cnn_model.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

print("Model and tokenizer saved successfully.")

# To load the model and tokenizer later, you can use:
# loaded_model = tf.keras.models.load_model('sentiment_cnn_model.h5')
# with open('tokenizer.pickle', 'rb') as handle:
#     loaded_tokenizer = pickle.load(handle)

Text: I feel like nothing is going right for me today.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I just can't seem to focus on anything right now.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I'm really struggling to keep up, and it's frustrating.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user might be depressed. Confidence: 1.00

Text: Today feels like a complete waste; I can’t get anything done.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Prediction: The user is in a positive mood. Confidence: 1.00

Text: I'm overwhelmed, and it's dragging my mood down.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user might be depressed. Confidence: 0.93

Text: I feel stuck in a rut, and it’s wearing me out.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Prediction: The user might be depressed. Confidence: 1.00

Text: Everything I touch seems to go wrong today.
1/1 ━━━━━━━━━━

Prediction: The user is in a positive mood. Confidence: 1.00

Model and tokenizer saved successfully.


In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout

# Define your vocabulary size and maximum sequence length
max_words = 10000  # or whatever your vocabulary size is
max_len = 100  # or whatever your maximum sequence length is

# Create the CNN model
embedding_dim = 100

model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now print the model summary
print(model.summary())

# Train the model (you may need to increase the number of epochs)
history = model.fit(X_train_pad, y_train_cat,
                    epochs=20,
                    batch_size=32,
                    validation_split=0.1,
                    verbose=1)

# Function to predict sentiment with temperature scaling
def predict_sentiment(text, temperature=1.0):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    
    # Apply temperature scaling
    scaled_prediction = np.exp(np.log(prediction[0]) / temperature)
    scaled_prediction /= np.sum(scaled_prediction)
    
    predicted_class = np.argmax(scaled_prediction)
    confidence = scaled_prediction[predicted_class]
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a negative mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations
examples = [
    "I feel so hopeless and sad all the time.",
    "I'm excited about my new job and looking forward to the future.",
    "I don't know if life is worth living anymore.",
    "I had a great day with my friends and family.",
    "This weather is terrible, and I'm having a bad day.",
    "I'm feeling a bit down today, but I'm sure things will get better."
]

for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example, temperature=1.5)}\n")

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.9023 - loss: 0.4012 - val_accuracy: 0.9408 - val_loss: 0.1681
Epoch 2/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9566 - loss: 0.1270 - val_accuracy: 0.9605 - val_loss: 0.1162
Epoch 3/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9870 - loss: 0.0464 - val_accuracy: 0.9642 - val_loss: 0.1214
Epoch 4/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9935 - loss: 0.0233 - val_accuracy: 0.9525 - val_loss: 0.1506
Epoch 5/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9967 - loss: 0.0178 - val_accuracy: 0.9620 - val_loss: 0.1699
Epoch 6/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9977 - loss: 0.0136 - val_accuracy: 0.9547 - val_loss: 0.1999
Epoch 7/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.9986 - loss: 0.0083 - val_accuracy: 0.9620 - val_loss: 0.2170
Epoch 8/20
385/385 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - accuracy: 0.9978 - loss: 0.0086 - va

C:\Users\shrir\AppData\Local\Temp\ipykernel_9648\3616276600.py:39: RuntimeWarning: divide by zero encountered in log
  scaled_prediction = np.exp(np.log(prediction[0]) / temperature)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Prediction: The user is in a negative mood. Confidence: 1.00

Text: I'm feeling a bit down today, but I'm sure things will get better.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Prediction: The user is in a positive mood. Confidence: 1.00



In [55]:
def predict_sentiment(text, temperature=1.0):
    sequence = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded)
    
    # Apply temperature scaling
    scaled_prediction = np.exp(np.log(prediction[0]) / temperature)
    scaled_prediction /= np.sum(scaled_prediction)
    
    predicted_class = np.argmax(scaled_prediction)
    confidence = scaled_prediction[predicted_class]
    
    if predicted_class == 2:
        return f"The user might be depressed. Confidence: {confidence:.2f}"
    elif predicted_class == 1:
        return f"The user is in a negative mood. Confidence: {confidence:.2f}"
    else:
        return f"The user is in a positive mood. Confidence: {confidence:.2f}"

# Test the model with some example conversations

examples = [
        "I feel so hopeless and sad all the time.",
        "I'm excited about my new job and looking forward to the future.",
        "I don't know if life is worth living anymore.",
        "I had a great day with my friends and family.",
        "This weather is terrible, and I'm having a bad day.",
        "I'm feeling a bit down today, but I'm sure things will get better.",
        "I'm really proud of what I accomplished today.",
        "Sometimes I wonder if I'll ever find my way.",
        "I had a productive meeting, but I still have a lot on my plate.",
        "Today was just another ordinary day.",
        "I'm thrilled about my upcoming vacation!",
        "I'm feeling overwhelmed by everything that’s happening.",
        "I'm content with where I am in life right now.",
        "I can't shake off this feeling of dread.",
        "I had a fun time at the park with my friends last weekend."
    ]
for example in examples:
    print(f"Text: {example}")
    print(f"Prediction: {predict_sentiment(example, temperature=1.5)}\n")

Text: I feel so hopeless and sad all the time.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: The user might be depressed. Confidence: 0.98

Text: I'm excited about my new job and looking forward to the future.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: The user is in a negative mood. Confidence: 1.00

Text: I don't know if life is worth living anymore.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Prediction: The user is in a negative mood. Confidence: 0.99

Text: I had a great day with my friends and family.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Prediction: The user is in a negative mood. Confidence: 1.00

Text: This weather is terrible, and I'm having a bad day.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Prediction: The user might be depressed. Confidence: 0.96

Text: I'm feeling a bit down today, but I'm sure things will get better.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Prediction: The user is in a negative mood. Confidence: 1.00

Text: I'm really proud of what I accomplished today.
1

In [11]:
!pip install keras

In [21]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Load data
df = pd.read_csv('merged_output.csv')

# Load positive and depressed words
with open('positive-words.txt', 'r') as file:
    positive_words = set(file.read().splitlines())
with open('negative-words.txt', 'r') as f:
    negative_words = set(f.read().splitlines())
with open('depressedword.txt', 'r') as file:
    depressed_words = set(file.read().splitlines())

# Normalize text function
def normalize_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Applying normalization
df['content'] = df['content'].apply(normalize_text)

# Label sentiment
def label_sentiment(text):
    words = set(text.split())
    positive_count = len(words.intersection(positive_words))
    negative_count = len(words.intersection(negative_words))
    depressed_count = len(words.intersection(depressed_words))
    
    if depressed_count > max(positive_count, negative_count):
        return 2  # Depressed
    elif negative_count > positive_count:
        return 1  # Negative
    else:
        return 0  # Positive

# Create labels based on word counts
df['label'] = df['content'].apply(label_sentiment)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2, random_state=42)

# Tokenize the text
max_words = 670000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

# Encode labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Convert to categorical
y_train_cat = tf.keras.utils.to_categorical(y_train_encoded, num_classes=3)
y_test_cat = tf.keras.utils.to_categorical(y_test_encoded, num_classes=3)

# Model parameters
embedding_dim = 200

# Define the CNN model
model = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(128, 5, activation='relu'),
    BatchNormalization(),
    GlobalMaxPooling1D(),
    Conv1D(64, 5, activation='relu'),
    BatchNormalization(),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),  # Additional dense layer
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)  # Changed to .keras

# Train the model
history = model.fit(X_train_pad, y_train_cat,
                    validation_split=0.2,  # Use part of the training set for validation
                    epochs=10,  # You can adjust this number
                    batch_size=32,  # You can adjust this number
                    callbacks=[early_stopping, model_checkpoint],
                    verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test_cat)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_6          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_7          │ ?                      │   0 (unbuilt) │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/10


ValueError: Input 0 of layer "conv1d_9" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 128)

In [23]:
history = model.fit(X_train_pad, y_train_cat,
                    epochs=5,
                    batch_size=32,
                    validation_split=0.1,
                    verbose=1)

Epoch 1/5


AttributeError: 'NoneType' object has no attribute 'items'